## Parse date

In [ ]:
from datetime import datetime
from dateutil import tz
import re

def parse_date(date):
    if len(date.split('.')) > 1 :
        dates = date.split(' ')
        dates[1] = re.sub("\D", "", dates[1])
        dates[3] = dates[3].split('.')[0]
        date = " ".join(dates)
        date = datetime.strptime(date, '%B %d %Y, %H:%M:%S')
    else:
        date = date[date.find(" ")+1:]
        date = datetime.strptime(date, '%b %d %H:%M:%S %Y')

    tzinfo = tz.gettz('Asia/Taipei')
    return date.replace(tzinfo=tzinfo).isoformat()

dates = [
    'December 9th 2016, 22:24:35.000',
    'Wed Oct 5 20:53:22 2016'
]
[parse_date(date) for date in dates]

## Parse author

In [19]:
author = 'assggy (IamCarmelo)'
author.split(" ")[0]

'assggy'

## Mapping (Blog)

In [ ]:
# from datetime import datetime
# from elasticsearch_dsl import DocType, Date, Integer, Keyword, Text, connections

# # Define a default Elasticsearch client
# connections.create_connection(hosts=['13.78.14.166'])

# class Article(DocType):
#     title = Text(analyzer='snowball', fields={'raw': Keyword()})
#     body = Text(analyzer='snowball')
#     tags = Keyword()
#     published_from = Date()
#     lines = Integer()

#     class Meta:
#         index = 'blog'

#     def save(self, ** kwargs):
#         self.lines = len(self.body.split())
#         return super(Article, self).save(** kwargs)

#     def is_published(self):
#         return datetime.now() > self.published_from

# # create the mappings in elasticsearch
# Article.init()

# # create and save and article
# article = Article(meta={'id': 42}, title='Hello world!', tags=['test'])
# article.body = ''' looong text '''
# article.published_from = datetime.now()
# article.save()

# article = Article.get(id=42)
# print(article.is_published())

# # Display cluster health
# print(connections.get_connection().cluster.health())

## Mapping (Post)

In [ ]:
# from datetime import datetime
# from elasticsearch_dsl import DocType, Date, Nested, Boolean, \
#     analyzer, InnerDoc, Completion, Keyword, Text

# html_strip = analyzer('html_strip',
#     tokenizer="standard",
#     filter=["standard", "lowercase", "stop", "snowball"],
#     char_filter=["html_strip"]
# )

# class Comment(InnerDoc):
#     author = Text(fields={'raw': Keyword()})
#     content = Text(analyzer='snowball')
#     created_at = Date(default_timezone='Asia/Taipei')

#     def age(self):
#         return datetime.now() - self.created_at

# class Post(DocType):
#     title = Text()
#     title_suggest = Completion()
#     created_at = Date()
#     published = Boolean()
#     category = Text(
#         analyzer=html_strip,
#         fields={'raw': Keyword()}
#    )

# # 1  city.raw 字段是 city 字段的 keyword 类型字段
# # 2  city 字段将被当做 full text 进行搜索
# # 34 city.raw 可用于排序和聚合

#     comments = Nested(Comment)

#     class Meta:
#         index = 'blog'

#     def add_comment(self, author, content):
#         self.comments.append(Comment(author=author, content=content, created_at=datetime.now()))

#     def save(self, ** kwargs):
#         self.created_at = datetime.now()
#         return super().save(** kwargs)
    

# first = Post(title='My First Blog Post, yay!', published=True)
# first.category = ['everything', 'nothing']
# first.meta.id = 55
# first.add_comment('me', 'This is nice!')
# first.add_comment('me', 'dfsdfsf!')
# first.add_comment('me', 'asdfasdfasdfasd!')

# first.save()
# # first.update(published=True, published_by='me')



## Mapping (PTT)

In [4]:
from elasticsearch_dsl import connections, DocType, Date, Nested, InnerDoc, Keyword, Text, Ip, Integer
from datetime import datetime
connections.create_connection(hosts=['13.78.14.166'])

class Message(InnerDoc):
    push_tag = Keyword(ignore_above=256)
    push_userid = Keyword(ignore_above=256)
    push_content = Keyword(ignore_above=256)
    push_ipdatetime = Keyword(ignore_above=256)


class MessageCount(InnerDoc):
    all = Integer()
    count = Integer()
    controversial = Integer()
    push = Integer()
    boo = Integer()
    neutral = Integer()


class Article(DocType):
    article_id = Keyword(ignore_above=256)
    article_title = Text(
        analyzer='ik_max_word',
        search_analyzer = 'ik_max_word'
    )
    author = Keyword(ignore_above=256)
    author_parsed = Keyword(ignore_above=256)
    board = Keyword(ignore_above=256)
    content = Text(
        analyzer='ik_max_word',
        search_analyzer = 'ik_max_word'
    )
    date = Keyword(ignore_above=256)
    date_parsed = Date(default_timezone='Asia/Taipei')
    ip = Keyword(ignore_above=256)
    message_count = Nested(MessageCount)
    messages = Nested(Message)

    class Meta:
        index = 'ptt-2018-06'
        doc_type = 'article'

Article.init()
        
#     def add_message(self, author, content):
#         self.message.append(Message(author=author, content=content, created_at=datetime.now()))
        
#     def add_message_count(self, author, content):
#         self.message_count.append(MessageCount(author=author, content=content, created_at=datetime.now()))

#     def save(self, ** kwargs):
#         self.created_at = datetime.now()
#         return super().save(** kwargs)
# first = Post(title='My First Blog Post, yay!', published=True)
# first.category = ['everything', 'nothing']
# first.meta.id = 47
# first.add_comment('me', 'This is nice!')
# first.save()e
# # first.update(published=True, published_by='me')


In [ ]:
# from elasticsearch_dsl import Search
# connections.create_connection(hosts=['13.78.14.166'])
# client = Elasticsearch()

# s = Search(index="test2-2018-06") \
#     .filter("term", category="search") \
#     .query("match", title="感情")   \
#     .exclude("match", description="父母")

# s.aggs.bucket('per_tag', 'terms', field='tags') \
#     .metric('max_lines', 'max', field='lines')

# response = s.execute()

# for hit in response:
#     print(hit.meta.score, hit.title)

# for tag in response.aggregations.per_tag.buckets:
#     print(tag.key, tag.max_lines.value)

In [22]:
from elasticsearch_dsl import Search, connections
connections.create_connection(hosts=['13.78.14.166'])
board = "Boy-Girl"
board = "Gossiping"

# Term 精确
# Match 模糊
s = Search(index="test-2018-06") \
    .filter("term", board=board) \
    .query("match", content="西瓜")   \
#     .query("match", article_title="西瓜 夏天")\
#     .exclude("match", article_title="照片")

response = s.execute()
for hit in response:
    print(hit.meta.score, hit.article_title)


11.233568 [新聞] 西瓜也有身份證了 後龍西瓜節週六熱鬧登
11.233568 [新聞] 西瓜也有身份證了 後龍西瓜節週六熱鬧登
11.217221 [問卦] 有紅西瓜的西瓜汁 為什麼沒小玉西瓜汁？
11.177635 [新聞] 〈中部〉福興西瓜節週日登場 三選將齊促
11.175111 [問卦] 有紅西瓜的西瓜汁 為什麼沒小玉西瓜汁？
11.174166 Re: [問卦] 溪州有什麼特產啊
11.173001 Re: [問卦] 溪州有什麼特產啊
11.14826 [新聞] 〈中部〉福興西瓜節週日登場 三選將齊促
11.067176 Re: [新聞] 【選情初探】民進黨：攻下台北、新北　就
11.065167 Re: [新聞] 【選情初探】民進黨：攻下台北、新北　就


In [27]:
from elasticsearch_dsl import Search, connections
connections.create_connection(hosts=['13.78.14.166'])
# board = "Gossiping"
board = "Boy-Girl"
s = Search(index="ptt-2018-06") \
    .filter("term", board=board) \
    .query("match", article_title="哥哥")  \
#     .query("match", content="西瓜 夏天")\
#     .exclude("match", article_title="照片")

response = s.execute()
for hit in response:
    print(hit.message_count.count, hit.article_title, hit.author, hit.meta.score,  hit.date_parsed)

In [28]:
from elasticsearch_dsl import Search, connections
connections.create_connection(hosts=['13.78.14.166'])
board = "Boy-Girl"
board = "Gossiping"

# Term 精确
# Match 模糊
s = Search(index="ptt-2018-06") \
    .filter("term", board=board) \
    .query("match", content="西瓜")   \
#     .query("match", article_title="西瓜 夏天")\
#     .exclude("match", article_title="照片")

response = s.execute()
for hit in response:
    print(hit.message_count.count, hit.article_title, hit.author, hit.meta.score,  hit.date_parsed)


9 [新聞] 西瓜也有身份證了 後龍西瓜節週六熱鬧登 qqq87112 (kaim) 12.909994 2018-06-05T15:20:50+08:00
0 Re: [問卦] 溪州有什麼特產啊 kutkin (  ) 12.744474 2018-06-06T11:00:54+08:00
19 [問卦] 西瓜的真實身份是？ ysc1213 (ysc) 12.708532 2018-01-23T10:45:04+08:00
8 [問卦] 有紅西瓜的西瓜汁 為什麼沒小玉西瓜汁？ q0000000 (十方眾生皆蒙昧 道消魔長) 12.691474 2018-05-13T10:44:13+08:00
4 [新聞] 〈中部〉福興西瓜節週日登場 三選將齊促 qqq87112 (kaim) 12.678584 2018-06-01T08:43:56+08:00
1 Re: [新聞] 【選情初探】民進黨：攻下台北、新北　就 sunyeah (   湯元嗎) 12.550674 2018-05-20T23:55:46+08:00
2 [問卦] 憾！小X百貨竟然沒賣西瓜刀？ Marzzze (Marzzze) 12.5249 2018-05-25T14:26:09+08:00
4 [新聞] 國宴西瓜行情佳　花警啟動「護瓜專案」 gjsjhang (臺灣杉－Biang) 12.500449 2018-05-10T13:48:46+08:00
3 [新聞] 馬英九叫賣西瓜 1顆1萬8800元助弱勢 cc9i (正直與善良) 12.471011 2018-05-19T18:26:17+08:00
3 [新聞] 猥褻無極限！爆乳女模開腿露下體破西瓜　 zuvupa (阿嘉) 12.466688 2018-04-26T15:02:44+08:00
